# Справочники для мониторинга рекламы в интернете
Пример ноутбука для работы со справочниками по мониторингу рекламы в интернете.

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter 

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from pathlib import Path
from IPython.display import JSON

from mediascope_api.core import net as msnet
from mediascope_api.crossweb import catalogs as cwc
from mediascope_api.crossweb import tasks as cwts


mnet = msnet.MediascopeApiNetwork()
cats = cwc.CrossWebCats()

# Доступные атрибуты для параметров

При формировании задания можно указать следующие параметры:

- **filters** - Фильтры
- **slices** - Срезы
- **statistics** - Статистики
- **scales** - Шкала, если задана статистика со шкалой (reachN)

Доступные атрибуты и допустимые для них значения можно получить методом `get_monitoring_unit()`:

In [ ]:
cats.get_monitoring_unit()

# Доступный период данных

Узнать доступный период данных можно после выполнения ячейки ниже

In [ ]:
cats.get_date_range(ids=[3]) 

# Состав проекта по мониторингу рекламы в интернете

Видеореклама, креатив которой размещен на url по определенным маскам (Web Desktop)

adSourceTypeId = 2; 
useTypeId = 1 

# Социально-демографические и географические переменные

Получим список всех переменных, доступных в проекте

In [ ]:
df_demo = cats.get_property()
df_demo

Экспорт справочника в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
demo_cat = cats.get_property()
writer = pd.ExcelWriter('demo_cat.xlsx')
demo_cat.to_excel(writer, 'demo_cat', index=False)
writer.save()

### Важно!
Demo и geo переменные, которые используются в заданиях, должны быть прописаны строго в следующем формате: 

__demoVariable__

То есть первое слово в переменной должно быть написано со строчной буквы, второе - с заглавной. Пример: __ageGroup__

### Поиск переменных

Поиск переменных осуществляется с помощью функции `cats.find_property()`, которая находит все переменные, содержащие заданный текст как в самом ее названии, так и в ее названии для API (поля 'name' и 'entityTitle'). 

Если нужно найти переменную без входящих в нее категорий, то необходимо передать в эту функцию дополнительный параметр

```
    expand=False
```

Если нужно найти переменную с входящими в нее категориями, то можно не передавать параметр expand. В таком случае для него автоматически проставится значение True.

Найдем переменные, которые содержат текст "Пол"

In [ ]:
cats.find_property('Пол', expand=True)

Рассмотрим географические переменные.

Численность населенного пункта (cityPop) и численность населенного пункта 100k+ или 100k- (cityPop100)

In [ ]:
cats.find_property('cityPop', expand=True)

Федеральный округ (federalOkrug)

In [ ]:
cats.find_property('federalOkrug', expand=True)

# Медиа справочник

Получим весь медиа справочник и отобразим первые пять записей.

### Важно!

**Загрузка всего медиа справочника может потребовать много памяти и времени. Для экономии памяти и ускорения загрузки, пожалуйста, пользуйтесь поиском по медиа справочнику.**

In [ ]:
df_media = cats.get_media()
df_media.head()

Экспорт медиа справочника в Excel. Файл сохраняется в директорию, в которой лежат текущие ноутбуки.

In [ ]:
media_cat = cats.get_media()
writer = pd.ExcelWriter('media_cat.xlsx')
media_cat.to_excel(writer, 'media_cat', index=False)
writer.save()

**В Cross Web трехуровневая иерархия медиа объектов:**

- **Холдинг (holding)** = Холдинг
- **Ресурс (resource)** = Проект/Сайт
- **Медиа продукт (product)** = Сабсекция

**Холдинг** объединяет ресурсы, которые юридически принадлежат одной компании. 

**Ресурс** объединяет в себе веб аудиторию (десктопный, мобильный веб) и приложения. 

**Медиа продукт** - это раздел в десктопном и мобильном вебе или приложение. Разделы в десктопном и мобильном вебе определяются регулярным выражением.

**В Cross Web тематики определяются двумя способами:**

**- тематики для медиа продуктов (theme)**

Таким образом, разные разделы одного ресурса могут относиться к разным тематикам.

**- тематики для ресурсов (resourceTheme)**

В этом случае для всего ресурса выбирается только одна наиболее подходящая тематика, даже если ресурс состоит из медиа продуктов, относящихся к различным тематикам.

### Холдинги
Получим список холдингов

In [ ]:
df_holdings = cats.get_holding()
df_holdings

### Ресурсы
Получим список ресурсов и отобразим первые пять записей

In [ ]:
df_resources = cats.get_resource()
df_resources.head()

### Медиа продукты
Получим список медиа продуктов и отобразим первые пять записей

In [ ]:
df_products = cats.get_product()
df_products.head()

### Тематики для медиа продуктов
Получим список тематик для медиа продуктов и отобразим первые пять записей

In [ ]:
df_thematics = cats.get_theme()
df_thematics.head()

### Тематики для ресурсов
Получим список тематик для ресурсов и отобразим первые пять записей

In [ ]:
df_resource_thematics = cats.get_resource_theme()
df_resource_thematics.head()

### Поиск в медиа справочнике

Поиск в медиа справочнике осуществляется с помощью следующих методов: 

- **get_media()** - отображает все поля медиа справочника; 
- **get_holding()** - отображает название холдинга и его id; 
- **get_resource()** - отображает название ресурса и его id; 
- **get_product()** - отображает название медиа продукта и его id; 
- **get_theme()** - отображает название тематики для медиа продуктов и ее id;
- **get_resource_theme()** - отображает название тематики для ресурсов и ее id.

Для всех перечисленных выше методов поддерживается поиск по следующим параметрам:

- **holding** - поиск по названию холдинга. Возможно задавать часть названия.
- **holding_ids** - поиск по cписку идентификаторов холдингов.
- **resource** - поиск по названию ресурса. Возможно задавать часть названия.
- **resource_ids** - поиск по cписку идентификаторов ресурсов.
- **product** - поиск по названию медиа продукта. Возможно задавать часть названия.
- **product_ids** - поиск по cписку идентификаторов медиа продуктов.
- **theme** - поиск по названию тематики для медиа продуктов. Возможно задавать часть названия.
- **theme_ids** - поиск по cписку идентификаторов тематик для медиа продуктов.
- **resource_theme** - поиск по названию тематики для ресурсов. Возможно задавать часть названия.
- **resource_theme_ids** - поиск по cписку идентификаторов тематик для ресурсов.


#### Поиск по названию
Найдем холдинг "Авито" в медиа справочнике и выведем всю информацию по его медиа объектам

In [ ]:
df_avito = cats.get_media(holding='Авито')
df_avito

#### Поиск по идентификатору
Найдем все медиа объекты с идентификаторами тематик для медиа продуктов
- id 20 - Почта
- id 42 - Работа

In [ ]:
df_media_by_id = cats.get_media(theme_ids=[20, 42])
df_media_by_id

#### Поиск по нескольким параметрам

Параметры поиска можно комбинировать. 

Добавим к предыдущему примеру еще один параметр поиска - название ресурса Avito. 

In [ ]:
df_media_by_id_resource = cats.get_media(theme_ids=[20, 42], resource='Avito')
df_media_by_id_resource

# Справочники атрибутов описания креативов

### Поиск

Поиск в справочнике атрибутов описания креативов осуществляется с помощью следующих методов: 

- **get_monitoring()** - отображает название категорий товаров и услуг, рекламодателя, бренда, суббренда, модели и их id;
- **get_product_category_tree()** - отображает название категорий товаров и услуг и их id;  
- **get_product_model()** - отображает название модели и ее id;  
- **get_product_subbrand()** - отображает название суббренда и его id;
- **get_product_brand()** - отображает название бренда и его id;
- **get_product_advertiser()** - отображает название рекламодателя и его id;
- **get_product_category_l4()** - отображает название категории товаров и услуг 4-ого уровня и ее id;
- **get_product_category_l3()** - отображает название категории товаров и услуг 3-его уровня и ее id;
- **get_product_category_l2()** - отображает название категории товаров и услуг 2-ого уровня и ее id;
- **get_product_category_l1()** - отображает название категории товаров и услуг 1-ого уровня и ее id.

Для всех перечисленных выше методов поддерживается поиск по следующим параметрам без учета регистра:

- **product_model** - поиск по названию модели. Возможно задавать часть названия;
- **product_model_eng** - поиск по названию модели на английском языке. Возможно задавать часть названия;
- **product_model_ids** - поиск по списку идентификаторов моделей;
- **product_subbrand** - поиск по названию суббренда. Возможно задавать часть названия;
- **product_subbrand_eng** - поиск по названию суббренда на английском языке. Возможно задавать часть названия;
- **product_subbrand_ids** - поиск по списку идентификаторов суббрендов;
- **product_brand** - поиск по названию бренда. Возможно задавать часть названия;
- **product_brand_eng** - поиск по названию бренда на английском языке. Возможно задавать часть названия;
- **product_brand_ids** - поиск по списку идентификаторов брендов;
- **advertiser** - поиск по названию рекламодателя. Возможно задавать часть названия;
- **advertiser_eng** - поиск по названию рекламодателя на английском языке. Возможно задавать часть названия;
- **advertiser_ids** - поиск по списку идентификаторов рекламодателей;
- **product_category_l4** - поиск по названию категории товаров и услуг 4-ого уровня. Возможно задавать часть названия;
- **product_category_l4_eng** - поиск по названию категории товаров и услуг 4-ого уровня на английском языке. Возможно задавать часть названия;
- **product_category_l4_ids** - поиск по списку идентификаторов категории товаров и услуг 4-ого уровня;
- **product_category_l3** - поиск по названию категории товаров и услуг 3-его уровня. Возможно задавать часть названия;
- **product_category_l3_eng** - поиск по названию категории товаров и услуг 3-его уровня на английском языке. Возможно задавать часть названия;
- **product_category_l3_ids** - поиск по списку идентификаторов категории товаров и услуг 3-его уровня;
- **product_category_l2** - поиск по названию категории товаров и услуг 2-ого уровня. Возможно задавать часть названия;
- **product_category_l2_eng** - поиск по названию категории товаров и услуг 2-ого уровня на английском языке. Возможно задавать часть названия;
- **product_category_l2_ids** - поиск по списку идентификаторов категории товаров и услуг 2-ого уровня;
- **product_category_l1** - поиск по названию категории товаров и услуг 1-ого уровня. Возможно задавать часть названия;
- **product_category_l1_eng** - поиск по названию категории товаров и услуг 1-ого уровня на английском языке. Возможно задавать часть названия;
- **product_category_l1_ids** - поиск по списку идентификаторов категории товаров и услуг 1-ого уровня.


Параметры поиска можно комбинировать. 

#### Поиск по названию

Найдем категорию товаров и услуг 4-ого уровня "Маркетплейсы" в справочнике и выведем всех рекламодателей, относящихся к этой категории.

In [ ]:
df_find_marketplace = cats.get_product_advertiser(product_category_l4="маркетплейс")
df_find_marketplace

In [ ]:
writer = pd.ExcelWriter('adv_cat.xlsx')
df_find_marketplace.to_excel(writer, 'adv_cat', index=False)
writer.save()

#### Поиск по идентификатору

Найдем все бренды с идентификаторами категории товаров и услуг 2-ого уровня:
- id 2277 - УСЛУГИ СТРАХОВЫЕ
- id 2272 - УСЛУГИ ФИНАНСОВЫЕ

In [ ]:
df_brand_by_id = cats.get_product_brand(product_category_l2_ids=[2277, 2272])
df_brand_by_id

#### Поиск по нескольким параметрам

Параметры поиска можно комбинировать. 

Добавим к предыдущему примеру еще один параметр поиска - название рекламодателя СБЕР.

In [ ]:
df_brand_by_id_2 = cats.get_product_brand(product_category_l2_ids=[2277, 2272], advertiser='СБЕР') 
df_brand_by_id_2

# Справочники атрибутов событий рекламы

### Рекламные сети
Получим список рекламных сетей

In [ ]:
df_networks = cats.get_ad_network()
df_networks

### Рекламные плееры
Получим список рекламных серверов

In [ ]:
df_players = cats.get_ad_player()
df_players